In [ ]:
# !git clone https://github.com/soumitrapy/templatecv.git project
# %cd project
# !wget https://storage.googleapis.com/wandb_datasets/nature_12K.zip
# !unzip nature_12K.zip -d data/

In [ ]:
# !git pull

In [1]:
import wandb
import yaml
import os
from datetime import datetime

import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

import torch
from torch.utils.data import Dataset,DataLoader, random_split

from torchvision import transforms
#from torchvision.io import read_image

In [2]:
import yaml
config = yaml.safe_load(open("config/default.yaml"))
#config = yaml.safe_load(open("config/pretrained.yaml"))
#config = yaml.safe_load(open("config/smallcnn.yaml"))

# config['dataset']['batch_size']=32
# config['train']['epochs']=1
config


{'project': 'templatecv',
 'use_wandb': True,
 'dataset': {'name': 'CustomDataset',
  'path': './data/inaturalist_12K/',
  'img_size': 256,
  'class_names': ['Plantae',
   'Mammalia',
   'Animalia',
   'Reptilia',
   'Amphibia',
   'Aves',
   'Fungi',
   'Arachnida',
   'Mollusca',
   'Insecta'],
  'batch_size': 5},
 'model': {'name': 'defaultmodel',
  'in_channels': 3,
  'num_classes': 10,
  'filters': 4,
  'kernel_size': 3,
  'dense_neurons': 100},
 'train': {'epochs': 2, 'val_interval': 2}}

In [3]:
import wandb
if config.get('use_wandb',False):
    wandb.login()

wandb: Using wandb-core as the SDK backend.  Please refer to https://wandb.me/wandb-core for more information.
wandb: Currently logged in as: soumitrapy (soumitrapy-iit-madras) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


### DataLoader Creation

In [4]:
from preprocessing import CustomDataset
from torchvision import transforms
import torch

# Data augmentation and normalization for training
# Just normalization for validation
cfg = config['dataset']

data_transforms = {
    'train': transforms.Compose([
        transforms.RandomResizedCrop(cfg['img_size']),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
    'val': transforms.Compose([
        transforms.RandomResizedCrop(cfg['img_size']),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ]),
}
splits = ['train', 'val']
datasets = {x:CustomDataset(path= cfg['path']+x,
                            class_names=cfg['class_names'],
                            transform=data_transforms[x])
            for x in splits}

dataloaders = {x: torch.utils.data.DataLoader(datasets[x],
                                              batch_size=cfg['batch_size'],
                                              shuffle=True
                                              )
                for x in splits}

class_names = datasets['train'].class_names
print(f"train len: {len(datasets['train'])}, val len: {len(datasets['val'])}")

# We want to be able to train our model on an `accelerator <https://pytorch.org/docs/stable/torch.html#accelerators>`__
# such as CUDA, MPS, MTIA, or XPU. If the current accelerator is available, we will use it. Otherwise, we use the CPU.

device = torch.device('cuda' if torch.cuda.is_available() else "cpu")
print(f"device = {device}")

train len: 9999, val len: 2000
device = cpu


In [5]:
# # transform = transforms.Compose([
# #         transforms.Resize(tuple(config['dataset']['img_size'])),
# #         transforms.ToTensor()
# #     ])
# data_transforms = {
#     'train': transforms.Compose([
#         transforms.RandomResizedCrop(224),
#         transforms.RandomHorizontalFlip(),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
#     'val': transforms.Compose([
#         transforms.Resize(256),
#         transforms.CenterCrop(224),
#         transforms.ToTensor(),
#         transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
#     ]),
# }
# target_transform = None
# train_path = config['dataset']['path']+"train"
# test_path = config['dataset']['path']+"val"

# trainds = CustomDataset(path=train_path,
#                         class_names=config['dataset']['class_names'],
#                         transform=data_transforms['train'],
#                         target_transform=target_transform
#                         )

# val_split = int(0.2 * len(trainds))
# trainds, valds = random_split(trainds, [len(trainds) - val_split, val_split])
# testds = CustomDataset(path=test_path,
#                         class_names=config['dataset']['class_names'],
#                         transform=data_transforms['val'],
#                         target_transform=target_transform
#                         )
# traindl = DataLoader(trainds, batch_size=config['dataset']['batch_size'])
# valdl = DataLoader(valds, batch_size=config['dataset']['batch_size'])
# testdl = DataLoader(testds, batch_size=config['dataset']['batch_size'])
# print(len(trainds), len(valds), len(testds))

In [6]:
for x, y in dataloaders['train']:
    print(x.shape,y.shape)
    break

torch.Size([5, 3, 256, 256]) torch.Size([5])


### Model

In [7]:
from models import DefaultModel
#from models.smallcnncnn import SmallCNN

cfg = config['model']
model = DefaultModel(cfg)
model(x).shape

torch.Size([5, 10])

### Pretrained Model

In [ ]:
from models.pretrained import get_model
model = get_model(config['model'])
for param in model.parameters():
    param.requires_grad = False
for param in model.fc.parameters():
    param.requires_grad = True

In [8]:
loss_fn = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr = 0.001)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.01)

### Training Loop

In [11]:
from train import train
ru = None
if config['use_wandb']:
    wandb.init(
        project=config['project'],
        name = type(model).__name__+str(datetime.now())[8:18],
        config=config['model'],
    )
train(model=model, optimizer=optimizer, loss_fn=loss_fn, dataloaders=dataloaders,config=config['train'], model_config=config['model'], scheduler = scheduler, device = device, use_wandb = config['use_wandb'])

Epoch 1:  25%|██▌       | 507/2000 [02:11<06:25,  3.87it/s, train_accuracy=15.5, train_loss=3.49]


KeyboardInterrupt: 

In [ ]:
model_artifact = run.use_artifact("trained-model:latest")
model_dir = model_artifact.download()
model_path = os.path.join(model_dir, "trained_model.pt")
model_config = model_artifact.metadata

model = DefaultModel(**model_config)
model.load_state_dict(torch.load(model_path))
model.to(device)

    loss, accuracy, highest_losses, hardest_examples, true_labels, preds = evaluate(model, test_loader)

    run.summary.update({"loss": loss, "accuracy": accuracy})

    wandb.log({"high-loss-examples":
        [wandb.Image(hard_example, caption=str(int(pred)) + "," +  str(int(label)))
            for hard_example, pred, label in zip(hardest_examples, preds, true_labels)]})


### Load model

In [ ]:
# model_path = './checkpoints/ResNet_cuda2025-04-19 17_1.pth'
# model.load_state_dict(torch.load(model_path, map_location=device))

### Prediction

In [ ]:
from train import visualize_model
visualize_model(model,valdl, class_names, 100, device=device)